In [13]:
import numpy as np

from math import log2, sqrt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree
from sklearn import preprocessing
import matplotlib.pyplot as plt
import six
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz
from six import StringIO  
from IPython.display import Image  
import pydotplus
from tqdm.notebook import tqdm_notebook as tqdm
from sklearn.model_selection import GridSearchCV

from decision_trees import MyDecisionTree

dataset shape is (32560, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32560 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   39              32560 non-null  int64 
 1    State-gov      32560 non-null  object
 2    77516          32560 non-null  int64 
 3    Bachelors      32560 non-null  object
 4    13             32560 non-null  int64 
 5    Never-married  32560 non-null  object
 6    Adm-clerical   32560 non-null  object
 7    Not-in-family  32560 non-null  object
 8    White          32560 non-null  object
 9    Male           32560 non-null  object
 10   2174           32560 non-null  int64 
 11   0              32560 non-null  int64 
 12   40             32560 non-null  int64 
 13   United-States  32560 non-null  object
 14   <=50K          32560 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


ValueError: ignored

In [6]:
columns_names = ['age','workclass','fnlwg','education','education_num','martial_status',
                 'ocupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week',
                 'country','target']
df_train = pd.read_csv("adult_data.csv",header=None,names=columns_names)
df_train.drop(columns='fnlwg',inplace=True)
df_train.head()

,age,workclass,education,education_num,martial_status,ocupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
columns_names = ['age','workclass','fnlwg','education','education_num','martial_status',
                 'ocupation','relationship','race','sex','capital_gain','capital_loss','hours_per_week',
                 'country','target']
df_test = pd.read_csv("adult_test.csv",header=None,names=columns_names)
df_test.drop(columns='fnlwg',inplace=True)
df_test = df_test.iloc[1:]
df_test['age'] = pd.to_numeric(df_test['age'])
df_test.head()

,age,workclass,education,education_num,martial_status,ocupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
1,25,Private,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
2,38,Private,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
3,28,Local-gov,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
4,44,Private,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.
5,18,?,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K.


In [9]:
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:,-1]
label_enc = LabelEncoder()
y_train = label_enc.fit_transform(y_train)

X_test = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]
label_enc = LabelEncoder()
y_test = label_enc.fit_transform(y_test)

In [10]:
class RandomForest(object):
    def __init__(self, n_estimators=50, max_depth=None, max_features=0.7):
        self.n_estimators = n_estimators ## how many estimators we are going to use?
        self.max_depth = max_depth ## Max depth 
        self.max_features = max_features ## the percentage of the RANDOM features we are going to use
        self.bootstraps_row_indices = [] ## Which examples are in the current bootstrap
        self.feature_indices = [] ## Keeping track of features indices
        self.out_of_bag = [] ## To calculate the out of bag error
        
        decision_trees = []
        for i in range(n_estimators):
            decision_trees.append(MyDecisionTree(max_depth=max_depth)) ## Call our DT class
        
        self.decision_trees = decision_trees ## Our random forest
        
        ## OR  self.decision_trees = [MyDecisionTree(max_depth=max_depth) for i in range(n_estimators)]
        
    def _bootstrapping(self, num_training, num_features): ## _ is similar to private in other programming language
        """
        - INPUT : 
            num_training: how many training examples in this bootstrap
            num_features: how many features in this bootstrap
            
        - OUTPUT :
        - row_idx: the row indices corresponding to the row locations of the selected samples in the original dataset.
        - col_idx: the column indices corresponding to the column locations of the selected features  
                   in the original feature list.
        
        
        
        - Randomly select a sample dataset of size num_training with replacement from the original dataset. 
        - Randomly select certain number of features (num_features denotes the total number of features in X,
          without replacement from the total number of features.
        """ 
        
        sample_size = list(range(num_training))
        row_idx = np.random.choice(sample_size,num_training) # Create random row indices
        col_idx = np.random.permutation(num_features)[:int(num_features*self.max_features)] ## Permutation ( with replacement)
        return row_idx, col_idx
            
    def bootstrapping(self, num_training, num_features):
        """
        Initializing the bootstap datasets for each tree
        """
        
        for i in range(self.n_estimators):
            total = set(list(range(num_training))) ## we use a set to get the unique elements
            row_idx, col_idx = self._bootstrapping(num_training, num_features)
            total = total - set(row_idx) ## again we use a set. Subtract the row indices from the total
            self.bootstraps_row_indices.append(row_idx)
            self.feature_indices.append(col_idx)
            self.out_of_bag.append(total) ## total is used for the OOB 
            
            
    def fit(self, X, y):
        """
        Train decision trees using the bootstrapped datasets.
        """
        
        num_training, num_features = X.shape
        self.bootstrapping(num_training,num_features) ## initialize the bootstrapping
        for i in range((self.n_estimators)): ## loop over the trees 
            current_bootstraps_row_indices = self.bootstraps_row_indices[i]
            current_feature_indices = self.feature_indices[i]
            current_X = X[current_bootstraps_row_indices[:,np.newaxis], current_feature_indices] ## data for this tree
            current_y = y[current_bootstraps_row_indices]
            current_dt = self.decision_trees[i]
            current_dt.fit(current_X,current_y, 0) ## 0 for the initial depth
            print("Current Tree to fit : " ,i+1) ## Which tree we are using
            
            
    def OOB_score(self, X, y):
        """
        Calculate the OOB score
        """
        
        accuracy = []
        for i in range(len(X)): ## Loop over the full dataset
            predictions = []
            for t in range(self.n_estimators): ## loop over each decision tree
                if i in self.out_of_bag[t]: ## the data that is NOT used in the current tree
                    predictions.append(self.decision_trees[t].predict(X[i][self.feature_indices[t]])) ## Predict
            if len(predictions) > 0:
                accuracy.append(np.sum(predictions == y[i]) / float(len(predictions))) ## Majority voting
        return np.mean(accuracy) ## Total accuracy